In [ ]:
import pandas as pd
import numpy as np
import psycopg2

pd.set_option('display.max_columns', None)

In [ ]:
conn = psycopg2.connect(
        host="localhost",
        database="bloodmoneydb",
        user="postgres",
        password="password"
)

In [ ]:
sql_query = """
    SELECT *
    FROM joined_fight_event_fighters_data
"""

df = pd.read_sql_query(sql_query, con=conn)

Possible avenues for feature engineering and preprocessing:

* Encode whether a given fighter is local (city/state) or from the same country that is hosting an event

* Age at time of the fight

* Win streak (can encode a losing streak as a negative win streak)

* Define a way to measure the typical length of a fighter's fights

In [ ]:
def convert_event_date_and_fighter_dob_to_datetime(df, date_col='date', dob_col='date'):
    '''
    Convert date and dob columns to pandas datetime
    '''
    df[date_col] = pd.to_datetime(df[date_col])
    df[dob_col] = pd.to_datetime(df[dob_col])
    
    return df

def calculate_age_of_fighter(df, date_col='date', dob_col='dob'):
    '''
    Calculate the age of the fighter in years:
    Input:
        df: pd.DataFrame
        date_col: str (Datetime column name for event date)
        dob_col: str (Datetime column name for date of birth)
    Output:
        df: pd.DatFrame
            'age' column is an Int of the age of the fighter in years at the time of the event
    '''
    df = convert_event_date_and_fighter_dob_to_datetime(df=df, date_col='date', dob_col='dob')
    df['age'] = df['date'].dt.year - df['dob'].dt.year
    
    return df

The event data doesn't provide what state the event is in for USA and other countries. Without geotagging the cities to their respective states it isn't possible to expand the definition of local to be that of fighters from the same state as the event. Not sure if the juice is worth the squeeze here in figuring out how to geocode all the cities (including the international ones). This is one potential option to return to however depending on how relevant the theoretical home town effect is in the modeling. 

In [ ]:
def fighter_is_local(row):
    '''
    Define whether a given fighter is local which is whether the city of the event matches
    either the city they're either fighting out of or were born in
    '''
    if (row['city'] == row['foo_city']) | (row['city'] == row['born_city']):
        return True
    else:
        return False

    

def fighter_is_national(row):
    '''
    Define whether a given fighter's represented nation is the same nation hosting the event
    '''
    if (row['country'] == row['foo_country']) | (row['country'] == row['born_country']):
        return True
    else:
        return False

Come up with a measure of fight time that is standardized across fights of different potential length. Will need to be something along the lines of percent of total potential rounds fought. Will need to apply some sort of window function that calculates a rolling percentage of cumulative rounds fought vs. cumulative potential rounds. 

In [ ]:
def calculate_pct_of_possible_rounds_fought(df, fighter_id_col='fighter_id', date_col='date'):
    '''
    Calculate the percentage of scheduled rounds a fighter has fought:
    Input:
        df: pd.DataFrame
        fighter_id_col: str (column name for the fighter_id column)
        date_col: str (Datetime column name for event date)
    Output:
        df: pd.DatFrame
            'cumulative_possible_rds' column is an Int of the total schedules rounds
                in a fighters' career up to that point
            'cumulative_fought_rds' column is an Int of the total rounds a figher
                has fought up to that point
            'pct_rds_fought' column is a float of the percent of scheduled rounds a fighter 
                has fought in their career
    '''
    df = df.sort_values([fighter_id_col, date_col])
    df['ending_round_num'] = pd.to_numeric(df['ending_round_num'], errors='coerce')
    df['possible_rds'] = pd.to_numeric(df['possible_rds'], errors='coerce')

    grouped_possible_rounds = df['possible_rds'].groupby(df['fighter_id'])
    grouped_fought_rounds = df['ending_round_num'].groupby(df['fighter_id'])

    cumulative_possible_rounds = grouped_possible_rounds.cumsum()
    cumulative_fought_rounds = grouped_fought_rounds.cumsum()

    df['cumulative_possible_rds'] = cumulative_possible_rounds 
    df['cumulative_fought_rds'] = cumulative_fought_rounds
    df['pct_rds_fought'] = df['cumulative_fought_rds'] / df['cumulative_possible_rds']
    
    return df

In [ ]:
df = calculate_age_of_fighter(df=df)
df['is_local'] = df.apply(fighter_is_local, axis=1)
df['is_national'] = df.apply(fighter_is_national, axis=1)
df = calculate_pct_of_possible_rounds_fought(df=df)

Calculate the current win streak for each fighter.

In [ ]:
streak = df.sort_values(['fighter_id', 'date']).copy()

In [ ]:
streak['win_streak'] = 0

In [ ]:
streak['outcome'].value_counts()

In [ ]:
streak.reset_index(drop=True, inplace=True)

streak.head(n=10)

In [ ]:
previous_outcome = streak.head(n=8).groupby('fighter_id')['outcome'].apply(lambda x: x.shift(1, fill_value=np.nan))

In [ ]:
win_streak = streak.loc[0:7, 'win_streak']

In [ ]:
test_df = pd.DataFrame({'prev': previous_outcome, 'win_streak': win_streak})

In [ ]:
test_df

In [ ]:
ideal_output = [0, -1, 1, -1, -2, 1, -1, 1]

In [ ]:
def test_function(row):
    if row['prev'] is np.nan:
        return row['win_streak']
    elif (row['win_streak'] >= 0) & (row['prev'] == 'Win'):
        return row['win_streak'] + 1
    elif (row['win_streak'] < 0) & (row['prev'] == 'Win'):
        return 1
    elif (row['win_streak'] >= 0) & (row['prev'] == 'Loss'):
        return -1
    elif (row['win_streak'] < 0) & (row['prev'] == 'Loss'):
        return row['win_streak'] - 1

In [ ]:
streak_output = []

for index, row in test_df.iterrows():
    streak_output.append(test_function(row))

In [ ]:
print(streak_output, '\n', ideal_output)

Need to increment consecutive wins or losses appropriately. Probably best for now to just iterate through each row using the index of the dataframe as I would in R. Then I can post a code snippet and find the pythonic way to do it. 